# Publishing web maps and web scenes

An ArcGIS web map is an interactive display of geographic information through a composition of web layers, basemap and much more. A web scene is analogous to a web map but in the 3D space. To get and overview, visit the product documentation for [web maps](http://doc.arcgis.com/en/arcgis-online/reference/what-is-web-map.htm) and [web scenes](http://doc.arcgis.com/en/arcgis-online/reference/what-is-web-scene.htm).

This sample demonstrates how to create and publish simple examples of web maps and scenes using the Python API. If you are interested in learning more about the specification to author and publish complex and more illustrative maps, refer to this [documentation](https://developers.arcgis.com/web-map-specification/).

In [1]:
import os
import json
from IPython.display import display
import arcgis
from arcgis.gis import GIS
from arcgis.mapping import WebMap, WebScene

# connect to your GIS
# Create an anonymous connection to ArcGIS Online to search for contents
gis1 = GIS()
# Create a connection to your portal for publishing
gis2 = GIS("https://pythonapi.playground.esri.com/portal", "arcgis_python", "amazing_arcgis_123")

# Publish a web map
The ArcGIS API for Python at version **1.3** extends the `WebMap` class with the capability to author new web maps and edit existing ones. You can perform basic operations such as adding, removing layers.

In [2]:
# Create an empty web map with a default basemap
wm = WebMap()
wm.definition

{
  "operationalLayers": [],
  "baseMap": {
    "baseMapLayers": [
      {
        "url": "https://services.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer",
        "layerType": "ArcGISTiledMapServiceLayer",
        "resourceInfo": {
          "currentVersion": 10.3,
          "mapName": "Layers",
          "supportsDynamicLayers": false,
          "layers": [
            {
              "id": 0,
              "name": "Citations",
              "parentLayerId": -1,
              "defaultVisibility": false,
              "subLayerIds": "Portal for ArcGIS",
              "minScale": 0,
              "maxScale": 0
            }
          ],
          "tables": [],
          "spatialReference": {
            "wkid": 102100,
            "latestWkid": 3857
          },
          "singleFusedMapCache": true,
          "tileInfo": {
            "rows": 256,
            "cols": 256,
            "dpi": 96,
            "format": "JPEG",
            "compressionQuality": 90,
      

The above represents a template of a simple web map. This web map consists of a basemap web layer and an array of operational web layers. Notice the opertaional layer is empty without any web layer urls. We will search for a public web layer titled **2012 USA Median Age** published by **esri** account and apply that as an operational layer for this web map.

In [3]:
search_result = gis1.content.search("title:2012 USA Median AND owner:esri", 
                                   item_type = "Map Service", outside_org = True)

In [4]:
display(search_result)

[<Item title:"2012 USA Median Household Income (Mature Support)" type:Map Image Layer owner:esri>,
 <Item title:"2012 USA Median Net Worth (Mature Support)" type:Map Image Layer owner:esri>,
 <Item title:"2012 USA Median Home Value (Mature Support)" type:Map Image Layer owner:esri>,
 <Item title:"2012 USA Median Age (Mature Support)" type:Map Image Layer owner:esri>]

In [5]:
median_age_item = search_result[1] # pick the second search result
display(median_age_item)

<Item title:"2012 USA Median Net Worth (Mature Support)" type:Map Image Layer owner:esri>

### Add an operational layer to the web map
To add 'USA median age' web layer as an operational layer, you can call the `add_layer()` method and pass the `MapImageLayer` object.

In [6]:
#query the layers in the item
median_age_item.layers

[<Layer url:"https://server.arcgisonline.com/ArcGIS/rest/services/Demographics/USA_Median_Net_Worth/MapServer/0">,
 <FeatureLayer url:"https://server.arcgisonline.com/ArcGIS/rest/services/Demographics/USA_Median_Net_Worth/MapServer/1">,
 <FeatureLayer url:"https://server.arcgisonline.com/ArcGIS/rest/services/Demographics/USA_Median_Net_Worth/MapServer/2">,
 <FeatureLayer url:"https://server.arcgisonline.com/ArcGIS/rest/services/Demographics/USA_Median_Net_Worth/MapServer/3">,
 <FeatureLayer url:"https://server.arcgisonline.com/ArcGIS/rest/services/Demographics/USA_Median_Net_Worth/MapServer/4">]

In [7]:
for lyr in median_age_item.layers:
    wm.add_layer(lyr)

In [8]:
# query the layers added to the web map
for lyr in wm.layers:
    print(lyr.title)

Median Net Worth
Block Groups
Tracts
Counties
States


### Publish the web map as an item to the portal
Now that the web map content is ready, we will use the `save()` method to save the `WebMap` object as a web map item in the GIS. As parameters to the `save()` method, you need to specify some essential properties for the new web map item.

In [9]:
web_map_properties = {'title':'USA median age map',
                     'snippet':'This map service shows the median age of people' +\
                     'in the United States as of 2012 census. The Median Age for' +\
                      'the U.S. is 37 years of age.',
                     'tags':'ArcGIS Python API'}

# Call the save() with web map item's properties.
web_map_item = wm.save(item_properties=web_map_properties)
web_map_item

<Item title:"USA median age map" type:Web Map owner:arcgis_python>

### Display the web map
We have successfully published a web map with consisting of a basemap and desired web layer as the operational layer. We will use `arcgis.viz` module to read the published map as a `WebMap` object and interact with it.

In [10]:
web_map_obj = arcgis.mapping.WebMap(web_map_item)

# display the web map in an interactive widget
web_map_obj

MapView(hide_mode_switch=True, layout=Layout(height='400px', width='100%'))

The map by zooms to the extent of the content by default. You can interact with the widget and explore the spatial distribution of people's age.

# Publish a web scene
So far, we have seen how to publish a web map. In this section, we will observe how to publish a web scene. We will read a template web scene json from a text file, add an interesting 3D Scene Service as an operational layer. Then using the `add()` method, we will publish this as a web scene item.

In [11]:
# Read sample web scene json from text file. We use built-in json module to parse it into a Python dictionary
web_scene_dict = dict()
with open("data/web_scene_simple.json","r") as file_handle:
    web_scene_dict = json.load(file_handle)

display(web_scene_dict)

{'operationalLayers': [{'itemId': '',
   'title': '',
   'visibility': True,
   'opacity': 1,
   'url': '',
   'layerType': ''}],
 'baseMap': {'baseMapLayers': [{'id': '933075fa973f49948a88b84dae743704',
    'visibility': True,
    'opacity': 1,
    'layerDefinition': {},
    'url': 'http://services.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer',
    'layerType': 'ArcGISTiledMapServiceLayer'}],
  'title': 'World Street Map',
  'elevationLayers': [{'url': 'https://elevation3d.arcgis.com/arcgis/rest/services/WorldElevation3D/Terrain3D/ImageServer',
    'id': 'globalElevation_0',
    'layerType': 'ArcGISTiledElevationServiceLayer'}]},
 'spatialReference': {'wkid': 102100, 'latestWkid': 3857},
 'version': '1.4',
 'viewingMode': 'global',
 'tables': []}

The web scene above is pretty simple. Notice the `operationalLayers` array does not contain `url` to the service. Now we will search for a scene service titled **Montreal, Canada Buildings** published by **esri_3d** and apply that to the web scene.

In [12]:
search_result = gis1.content.search("title:Montreal, Canada Buildings AND owner:esri_3d", 
                                   item_type="scene service", outside_org = True)
display(search_result)

[<Item title:"Buildings_Montreal" type:Scene Layer owner:esri_3d>,
 <Item title:"Montreal, Canada Buildings" type:Scene Layer owner:esri_3d>]

In [13]:
buildings_layer = search_result[0]
display(buildings_layer)

<Item title:"Buildings_Montreal" type:Scene Layer owner:esri_3d>

In [14]:
# Update web scene's opertaional layer with properties of buildings_layer
web_scene_dict['operationalLayers'][0]['itemId'] = buildings_layer.itemid
web_scene_dict['operationalLayers'][0]['layerType'] = "ArcGISSceneServiceLayer"
web_scene_dict['operationalLayers'][0]['title'] = buildings_layer.title
web_scene_dict['operationalLayers'][0]['url'] = buildings_layer.url

### Publish the web scene as an item to the portal
We will use the `add()` from `ContentManager` class to create a web scene item. As parameters we will send a dictionary containing properties of the web scene item. The `text` property will contain the web scene dictionary updated in the previous step as a string.

In [15]:
web_scene_item_properties = {'title':'Web scene with photo realistic buildings',
                            'type':'Web Scene',
                            'snippet':'This scene highlights buildings of Montreal, Canada',
                            'tags':'ArcGIS Python API',
                            'text': json.dumps(web_scene_dict)}

# Use the add() method to publish a new web scenej
web_scene_item = gis2.content.add(web_scene_item_properties)
web_scene_item.share(True)
display(web_scene_item)

<Item title:"Web scene with photo realistic buildings" type:Web Scene owner:arcgis_python>

### Display the web scene
We have successfully published a web scene with consisting of a basemap, elevation layer and desired web layer as the operational layer. We will use `arcgis.mapping` module to read the published scene as a `WebScene` object and interact with it.

In [16]:
web_scene_obj = arcgis.mapping.WebScene(web_scene_item)

# display the interactive web scene in the notebook
web_scene_obj

MapView(hide_mode_switch=True, layout=Layout(height='400px', width='100%'), mode='3D')